In [67]:
import requests
from bs4 import BeautifulSoup

def check_robots_txt(url):

    robots_url = f"{url}/robots.txt"

    try:
        response = requests.get(robots_url)

        if response.status_code == 200:
            print(f"Status code: {response.status_code} (Success)")
            print("robots.txt Content:")
            print(response.text)
        else:
            print(f"Status Code: {response.status_code} (No robots.txt found)")

    except Exception as e :
        print(f"Error occurred: {e}")

check_robots_txt("http://lecture.pul.kr")

Status code: 200 (Success)
robots.txt Content:
User-agent: *
Disallow: /



In [79]:
import requests
from bs4 import BeautifulSoup
import time

def download_image(url, filename):
     response = requests.get(url)
    # if response.status_code == 200:
     with open(filename,'wb') as f:
        f.write(response.content)
         
url ='http://10.0.66.99/ex4.html'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

pdfs = soup.select('div.pdf_doc > .download-link')

for fn, pdf in enumerate(pdfs,1):

    src = pdf['href']
    download_pdf("http://10.0.66.99/ex4.html"+src, f'pdf_{fn:02}.pdf')
    time.sleep(2)

In [125]:
# 네이게이션 링크 추출 
# 글 링크 추출
import requests
from bs4 import BeautifulSoup
html='''
<nav class="navigation">
    <ul class="menu">
        <li><a href="https://example.com/home" class="nav-link">홈</a></li>
        <li><a href="https://example.com/about" class="nav-link">소개</a></li>
        <li><a href="https://example.com/contact" class="nav-link">연락처</a></li>
    </ul>
</nav>
<div class="content">
    <a href="https://example.com/article/1" class="article-link">첫 번째 글</a>
    <a href="https://example.com/article/2" class="article-link">두 번째 글</a>
</div>
<div class="footer">
    <a href="https://example.com/article/1" class="article-link">인기 많은 글</a>
    <a href="https://example.com/article/3" class="article-link">조회수 많은 글</a>
</div>
'''
soup = BeautifulSoup(html,"html.parser")

mylink = []

nav_links = soup.select('.nav-link')

for link in nav_links:
    mylink.append(link['href'])


article_links = soup.select('.article-link')

for link in article_links:
    mylink.append(link['href'])
print(mylink)
mylink = list(set(mylink))
print(mylink)



['https://example.com/home', 'https://example.com/about', 'https://example.com/contact', 'https://example.com/article/1', 'https://example.com/article/2', 'https://example.com/article/1', 'https://example.com/article/3']
['https://example.com/about', 'https://example.com/article/3', 'https://example.com/article/1', 'https://example.com/home', 'https://example.com/article/2', 'https://example.com/contact']


In [229]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
html_doc='''
<table class="data-table">
    <thead>
        <tr>
            <th>순위</th>
            <th>이름</th>
            <th>점수</th>
            <th>등급</th>
        </tr>
    </thead>
    <tbody>
        <tr class="student-row">
            <td class="rank">1</td>
            <td class="name">김철수</td>
            <td class="score">95</td>
            <td class="grade">A+</td>
        </tr>
        <tr class="student-row">
            <td class="rank">2</td>
            <td class="name">이영희</td>
            <td class="score">88</td>
            <td class="grade">B+</td>
        </tr>
    </tbody>
</table>
'''
soup = BeautifulSoup(html_doc,"html.parser")
headers = [th.text for th in soup.select ('thead th')]
data = []
rows = soup.select('tbody tr.student-row')
for row in rows:
    
    raw_data = [
        row.select_one('.rank').text,
        row.select_one('.name').text,
        row.select_one('.score').text,
        row.select_one('.grade').text
    ]

    # raw_data = [
    #     row.select_one('td')[0].text,
    #     row.select_one('td')[1].text,
    #     row.select_one('td')[2].text,
    #     row.select_one('td')[3].text
    # ]
     #raw_data = [td.text for td in row.select('td')]
    data.append(raw_data)

df = pd.DataFrame(data, columns=headers)
df.to_csv('student_scores.csv',index =False, encoding='utf-8')




In [273]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
html_doc='''
<form class="login-form">
    <div class="form-group">
        <label for="username">사용자명:</label>
        <input type="text" id="username" name="username" value="test_user" class="form-control">
    </div>
    <div class="form-group">
        <label for="email">이메일:</label>
        <input type="email" id="email" name="email" value="test@example.com" class="form-control">
    </div>
    <div class="form-group">
        <label>관심사:</label>
        <select name="interests" class="form-control">
            <option value="tech" selected>기술</option>
            <option value="science">과학</option>
            <option value="art">예술</option>
        </select>
    </div>
    <div class="form-group">
        <label>구독 여부:</label>
        <input type="checkbox" name="subscribe" checked class="form-check">
    </div>
</form>
'''

soup = BeautifulSoup(html_doc,"html.parser")
form_data = {}
# 입력필드 (택스트, 이메일)
inputs = soup.select("input[type='text'],input[type = 'email']")
for input in inputs:
    #print(input)
    #form_data[input['name']] = input['value']
    form_data[input['name']] = input.get('value','') # 안전한 값 가져오기 (기본값 지정)

# 선택된 옵션
select_opt = soup.select_one("select option[selected]") # 선택된 항목만
if  select_opt:
    select_ele = select_opt.find_parent('select') # 부모 접근
    form_data[select_ele['name']] = select_opt['value']

# 체크박스
checkbox = soup.select_one("input[type='checkbox']")
if checkbox:
    form_data[checkbox['name']] = checkbox.has_attr('checked') # 속성 존재 여부
                
form_data

# form_data

for k,v in form_data.items():
    print(f"#{k}: {v}")





username: test_user
email: test@example.com
interests: tech
subscribe: True


In [297]:
import requests
from bs4 import BeautifulSoup
html_doc='''
<div class="gallery">
    <div class="image-container">
        <img src="image1.jpg" alt="첫 번째 이미지" class="gallery-image" data-id="1">
        <div class="image-info">
            <span class="title">아름다운 풍경</span>
            <span class="photographer">촬영: 김작가</span>
        </div>
    </div>
    <div class="image-container">
        <img src="image2.jpg" alt="두 번째 이미지" class="gallery-image" data-id="2">
        <div class="image-info">
            <span class="title">도시의 밤</span>
            <span class="photographer">촬영: 이작가</span>
        </div>
    </div>
</div>
'''
soup = BeautifulSoup(html_doc,"html.parser")
images = []
for container in soup.select('.image-container'):
    img = container.select_one('img')
    info = container.select_one('.image-info')
    
    image_data = {
        'id' : img['data-id'],
        'src' : img['src'],
        'alt' : img['alt'],
        'title' : info.select_one('.title').text,
        'photographer': info.select_one('.photographer').text,
    
}

    images.append(image_data)
images


[{'id': '1',
  'src': 'image1.jpg',
  'alt': '첫 번째 이미지',
  'title': '아름다운 풍경',
  'photographer': '촬영: 김작가'},
 {'id': '2',
  'src': 'image2.jpg',
  'alt': '두 번째 이미지',
  'title': '도시의 밤',
  'photographer': '촬영: 이작가'}]